In [1]:
import sys
sys.path.append("..")
from azureopenaimanager.eam_prompts01 import *
from azureopenaimanager.eam_prompts03 import *

In [2]:
from azureopenaimanager.azureopenai_helper import AzureOpenAIManager

In [3]:
import os

from dotenv import load_dotenv
# Load environment variables for secrets
load_dotenv("../.env")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_KEY")
COLLECTION_NAME = "PWD_SENTENCE_TRANSFORMERS"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

In [5]:
azure_open_ai_manager = AzureOpenAIManager(
                        endpoint=AZURE_OPENAI_ENDPOINT,
                        api_key=AZURE_OPENAI_API_KEY,
                        deployment_id=AZURE_OPENAI_DEPLOYMENT_ID,
                        api_version=OPENAI_API_VERSION
                    )
    

In [6]:
question = "Get the locations and their hazards"

In [7]:
sql_query = None
context = eam_prompts_part_1
if context == "":
    msg,_,_,_ = azure_open_ai_manager.generate_answer_document(question)
else:
    msg,_,_,_ = azure_open_ai_manager.generate_answer_document_with_context(question,context)

In [8]:
msg

'locations, location_hazard, hazards'

In [9]:
msg_list = msg.split(",")

In [10]:
msg_list

['locations', ' location_hazard', ' hazards']

In [11]:
import json

def get_table_schema(table_name, json_path='../azureopenaimanager/eam_prompts.json'):
    """
    Returns the table schema (column definitions) for a given table name
    from the eam_prompts.json file.

    :param table_name: str, the name of the table (key in JSON)
    :param json_path: str, optional path to the JSON file
    :return: dict (schema) if key exists, else None
    """
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data.get(table_name)


In [12]:
table_schema = ""

In [13]:
for table_name in msg_list:
    table_schema = table_schema + " " + get_table_schema(table_name="locations")

In [14]:
context = eam_prompts_header + table_schema

In [15]:
msg,_,_,_ = azure_open_ai_manager.generate_answer_document_with_context(question,context)

In [16]:
from pprint import pprint
pprint(msg)

('```sql\n'
 'SELECT \n'
 '    l.location_name, \n'
 '    h.hazard_id, \n'
 '    h.hazard_description\n'
 'FROM \n'
 '    locations l\n'
 'JOIN \n'
 '    location_hazard lh ON l.location_name = lh.location_name\n'
 'JOIN \n'
 '    hazards h ON lh.hazard_id = h.hazard_id;\n'
 '```')
